In [33]:
# Import libraries
import pandas as pd
# Load & preprocess data
import sys
sys.path.append('Models')

# Tokenizer
from pyvi import ViTokenizer
# Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Model
from sklearn.linear_model import LogisticRegression
# Evaluation
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split



In [3]:
!pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 59.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.4 MB/s eta 0:00:00


In [34]:
data = pd.read_csv('/kaggle/input/vnfddataset/vn_news_223_tdlfr.csv')
train, test = train_test_split(data, test_size=0.3)



In [35]:
import pandas as pd  # type: ignore
import string


def preprocess_data(
    data,
    url=True,
    punctuation=True,
    lowercase=True,
    stopword=False,
    
):
    # Load stopwords
    with open('/kaggle/input/stopwords/vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
    # Function to remove stopwords

    def remove_stopwords(text):
        words = text.split()
        words = [word for word in words if word not in stopwords]
        return ' '.join(words)
    if url:
        # Remove URLs
        data['text'] = data['text'].str.replace(
            r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
    if punctuation:
        # Remove punctuation
        data['text'] = data['text'].str.replace(
            '['+string.punctuation+']', '', regex=True)
    if lowercase:
        # Lowercase
        data['text'] = data['text'].str.lower()
    if stopword:
        # Remove stopword
        data['text'] = data['text'].apply(remove_stopwords)

    return data

In [36]:
train.dropna(inplace=True)
train.drop_duplicates(subset='text', inplace=True, keep='first')

In [38]:
train = preprocess_data(train)

In [39]:
train['text'].dropna(inplace=True)

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [42]:

train['text'] = train['text'].astype(str).apply(ViTokenizer.tokenize)
X_train = train['text']
y_train = train['label']

In [43]:
# vectorizer = TfidfVectorizer()
# X_train = X_train.apply(' '.join)
# X_train = vectorizer.fit_transform(X_train)

train['text'] = train['text'].astype(str).apply(ViTokenizer.tokenize)
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train['text'])
y_train = train['label']

In [52]:
# Define the parameter grid
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
    'max_iter': list(range(100,800,100)),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    
}
# Create a StratifiedKFold object
stratified_kfold = StratifiedKFold(n_splits=30)
logreg=LogisticRegression()
# Create a GridSearchCV object

grid = GridSearchCV(logreg, param_grid, cv=stratified_kfold, refit=True, verbose=2, n_jobs=-1)
# Fit the model to the data
grid.fit(X_train, y_train)

# Print the best parameters
print(grid.best_params_)

Fitting 30 folds for each of 490 candidates, totalling 14700 fits


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

[CV] END C=0.001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END ....C=0.001, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ....C=0.001, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ....C=0.001, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ....C=0.001, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ....C=0.001, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ....C=0.001, max_iter=100, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=liblinear; total time=   0.0s
[CV] END C=0.001, max_iter=1

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END C=0.001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END C=0.001, max_iter=100, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END ....C=0.001, max_it

...

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

[CV] END .C=1000, max_iter=600, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .C=1000, max_iter=600, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .C=1000, max_iter=600, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .C=1000, max_iter=600, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .C=1000, max_iter=600, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .C=1000, max_iter=600, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .C=1000, max_iter=600, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .C=1000, max_iter=600, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .C=1000, max_iter=600, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .C=1000, max_iter=600, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .C=1000, max_iter=600, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .C=1000, max_iter=600, penalty=l1, solver=newton-cg; total time=   0.0s
[CV] END .C=1000, max_iter=6

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

{'C': 1000, 'max_iter': 200, 'penalty': 'l1', 'solver': 'saga'}


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [53]:
clf = grid.best_estimator_

In [54]:
# Training performance
y_train_pred = clf.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        91
           1       1.00      1.00      1.00        64

    accuracy                           1.00       155
   macro avg       1.00      1.00      1.00       155
weighted avg       1.00      1.00      1.00       155



In [55]:
# Evaluate the model on the test set
test = preprocess_data(test, url=True, punctuation=True, stopword=False)
X_test = vectorizer.transform(test['text'])
y_test = test['label']
# Make predictions on the test set
y_test_pred = clf.predict(X_test)
# Compute accuracy
print('Accuracy on the test set:', accuracy_score(y_test, y_test_pred))
# Compute F1 score
print(classification_report(y_test, y_test_pred))

Accuracy on the test set: 0.9104477611940298
              precision    recall  f1-score   support

           0       0.86      0.97      0.91        32
           1       0.97      0.86      0.91        35

    accuracy                           0.91        67
   macro avg       0.91      0.91      0.91        67
weighted avg       0.92      0.91      0.91        67

[CV] END .....C=1000, max_iter=700, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END .....C=1000, max_iter=700, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END .....C=1000, max_iter=700, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END .....C=1000, max_iter=700, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END .....C=1000, max_iter=700, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END .....C=1000, max_iter=700, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END .....C=1000, max_iter=700, penalty=l2, solver=lbfgs; total time=   0.1s
[CV] END .....C=1000, max_iter=700, penalty=l2, solver=lbfgs;